In [15]:
import numpy as np
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from sklearn.pipeline import Pipeline
from sktime.transformations.panel.rocket import MiniRocketMultivariate

from drowsiness_detection import config
from drowsiness_detection.models import ThreeDStandardScaler
from drowsiness_detection.run_grid_search_experiment import load_experiment_data
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit

In [16]:

config.set_paths(30, 10)

# load data
X_train, X_test, y_train, y_test, cv_splits = load_experiment_data(
    exclude_by="a",
    num_targets=2,
    seed=45,
    test_size=.2,
    split_by_subjects=True,
    use_dummy_data=False,
    nn_experiment=True,
    feature_col_indices=(5, 8, 9, 14, 15, 16, 19))


X_train shape: (28012, 300, 7), y_train shape: (28012,)
X_test shape: (7094, 300, 7), y_test shape: (7094,)


In [17]:
X_test = np.swapaxes(X_test, 1, 2)
X_train = np.swapaxes(X_train, 1, 2)

In [18]:
num_samples = None
X_test, y_test = X_test[:num_samples], y_test[:num_samples]
X_train, y_train = X_train[:num_samples], y_train[:num_samples]
print(X_test.shape)

(7094, 7, 300)


In [19]:
pipe = Pipeline(
    [
        ("scaler", ThreeDStandardScaler(feature_axis=1)),
        ("minirocket", MiniRocketMultivariate()),
        ("classifier", RidgeClassifier(alpha=0.001, normalize=True))
    ],
    verbose=True
)

In [20]:
search = GridSearchCV(estimator=pipe, param_grid=dict(classifier__alpha=[.1,1]), cv=PredefinedSplit(cv_splits), n_jobs=1, scoring="accuracy", verbose=1)

In [21]:
result = search.fit(X=X_train, y=y_train)

Fitting 1 folds for each of 2 candidates, totalling 2 fits
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   1.3s
[Pipeline] ........ (step 2 of 3) Processing minirocket, total= 2.2min


/home/tim/IM/venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), RidgeClassifier())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


[Pipeline] ........ (step 3 of 3) Processing classifier, total=  21.2s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   1.1s
[Pipeline] ........ (step 2 of 3) Processing minirocket, total= 2.0min


/home/tim/IM/venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), RidgeClassifier())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


[Pipeline] ........ (step 3 of 3) Processing classifier, total=  19.8s
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   1.4s
[Pipeline] ........ (step 2 of 3) Processing minirocket, total= 2.8min


/home/tim/IM/venv/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), RidgeClassifier())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


[Pipeline] ........ (step 3 of 3) Processing classifier, total=  27.0s


In [22]:
y_pred_train = search.best_estimator_.predict(X_train)
y_pred_test = search.best_estimator_.predict(X_test)

In [23]:
np.mean(y_train == y_pred_train)

0.8124375267742396

In [24]:
np.mean(y_test == y_pred_test)

0.5453904708204116

In [25]:
search.cv_results_

{'mean_fit_time': array([153.54056168, 139.05184436]),
 'std_fit_time': array([0., 0.]),
 'mean_score_time': array([48.02575421, 45.28586984]),
 'std_score_time': array([0., 0.]),
 'param_classifier__alpha': masked_array(data=[0.1, 1],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'classifier__alpha': 0.1}, {'classifier__alpha': 1}],
 'split0_test_score': array([0.66237079, 0.66948584]),
 'mean_test_score': array([0.66237079, 0.66948584]),
 'std_test_score': array([0., 0.]),
 'rank_test_score': array([2, 1], dtype=int32)}